In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.datasets import load_iris
import statistics
import itertools
import random
import math
import time
#from MDLP import MDLP_Discretizer


class MergedNode:
    def __init__(self):
        self.predicted_class = None
        self.feature_index = 0
        self.threshold = []
        self.children = []



class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.children = []


class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _best_split(self, X, y):
        m = y.size
        if m <= 1:
            return None, None
        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None
        for idx in range(self.n_features_):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()
            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_)
                )
                gini = (i * gini_left + (m - i) * gini_right) / m
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)
        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] <= thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr

                node.children.append(self._grow_tree(X_left, y_left, depth + 1))
                node.children.append(self._grow_tree(X_right, y_right, depth + 1))
        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] == node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class


    def mergeDecisionTrees(self,k,roots):
        flag = True
        for tree in roots:
            if tree.children:
                flag = False
                break

        if flag:
            labels = []
            for tree in roots:
                labels.append(tree.predicted_class)

            node = MergedNode()
            node.predicted_class = statistics.mode(labels)

            return node


        feature = []
        for tree in roots:
            if tree.children:
               feature.append(tree.feature_index)

        X_f = statistics.mode(feature)

        split_value = []

        for tree in roots:
            if X_f == tree.feature_index:
                split_value.append(tree.threshold)



        split_value = set(split_value)
        I_f = list(split_value)

        I_f.sort()



        pbranch = []
        branch = []
        for tree in roots:

            pbranch.append(self.computeBranch(tree, I_f, len(I_f) + 1, X_f))

        branch = np.array(pbranch).transpose().tolist()

        children = []

        for condition in range(len(I_f)+1):
            children.append(self.mergeDecisionTrees(k,branch[condition]))

        node = MergedNode()
        node.feature_index = X_f
        node.threshold = I_f
        node.children = children
        return node

    def computeBranch(self,root, I_f, len_I_f, X_f):

      #  I_f_new = len(I_f) + 1

        pbranch = []
        if not root.children :

            for condition in range(len_I_f):
                node = MergedNode()
                node.predicted_class = root.predicted_class
                #node.threshold = root.threshold
                #node.feature_index = root.feature_index
                #node.children.append(root.left, root.right)
                pbranch.append(node)
            return pbranch
        elif root.feature_index != X_f:
            child = []
            for val  in root.children:
                child.append(self.computeBranch(val, I_f, len_I_f, X_f))

            for condition in range(len_I_f):
                node = MergedNode()
                node.feature_index = root.feature_index
                node.threshold = root.threshold
                for index, val in enumerate(root.children):
                    node.children.append(child[index][condition])
                pbranch.append(node)
            return pbranch
        else:
            I_f1 = []
            I_f2 = []
            for val in I_f:
                if val < root.threshold:
                    I_f1.append(val)
                elif val == root.threshold:
                    I_f1.append(val)
                    I_f2.append(val)
                else:
                    I_f2.append(val)
            child = []
            if len(I_f1) == 1:
                child.append([])
                child[0].append(root.children[0])
            else:
                child.append(self.computeBranch(root.children[0], I_f1, len(I_f1), X_f))

            if len(I_f2) == 1:
                child.append([])
                child[1].append(root.children[1])
            else:
                child.append(self.computeBranch(root.children[1], I_f2, len(I_f2), X_f))

            for interval in range(len(I_f1)):
                pbranch.append(child[0][interval])

            for interval in range(len(I_f2)):
                pbranch.append(child[1][interval])

            if len(I_f1) >= 2 and len(I_f2) >= 2:
                if I_f1[-1] == I_f2[1] and I_f1[-2] == I_f2[0]:
                    node = MergedNode()
                    node.feature_index = root.feature_index
                    node.threshold = root.threshold
                    node.children.append(child[0][len(I_f1)])
                    node.children.append(child[1][len(I_f1)])
                    pbranch[len(I_f1)] = node

            return pbranch



dataset = datasets.load_iris()
X, y = dataset['data'], dataset['target']
feature_names, class_names = dataset['feature_names'], dataset['target_names']


#Discretization of data
numeric_features = np.arange(X.shape[1])  # all fetures in this dataset are numeric. These will be discretized
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state= 57)
'''discretizer = MDLP_Discretizer(features=numeric_features)

discretizer.fit(X_train, y_train)
X_train_discretized = discretizer.transform(X_train)

discretizer.fit(X_test, y_test)
X_test_discretized = discretizer.transform(X_test)'''


# Partitioning of the data
k = int(input("Enter number of distributed parties="))
subset_data =  np.array_split(X_train,k)
subset_label = np.array_split(y_train,k)

roots = []
feature = []
thresholds = []
results = []

for i in range(k):
    clf = DecisionTreeClassifier(max_depth=2)
    start = time.time()
    clf.fit(subset_data[i],subset_label[i])
    end = time.time()
    print(f"Time for constructing tree {i} is {end-start}")
    roots.append(clf.tree_)
    feature.append(clf.tree_.feature_index)
    thresholds.append(clf.tree_.threshold)
#   results.append(clf.predict(X_test_discretized))


start = time.time()
final_node = clf.mergeDecisionTrees(k, roots)
end = time.time()
print(f"Time for merging the trees is {end-start}")

def merged_prediction(root,inputs):
    if not root.children:
        return root.predicted_class
    for index,threshold in enumerate(root.threshold):
        if inputs[root.feature_index] <= threshold:
            return merged_prediction(root.children[index],inputs)

        return merged_prediction(root.children[index+1],inputs)

start = time.time()
ans = []
for row in X_test:
    ans.append(merged_prediction(final_node,row))
end = time.time()

print(f" Prediction Time by the merged tree is {end-start}")

count = 0
for i in range(len(ans)):
    if y_test[i] == ans[i]:
        count += 1

accuracy = count/len(y_test)
print(f"Accuracy of the Merged tree is {accuracy}")


Enter number of distributed parties=4
Time for constructing tree 0 is 0.0021524429321289062
Time for constructing tree 1 is 0.0023148059844970703
Time for constructing tree 2 is 0.0019173622131347656
Time for constructing tree 3 is 0.0020132064819335938
Time for merging the trees is 0.0007379055023193359
 Prediction Time by the merged tree is 0.00035953521728515625
Accuracy of the Merged tree is 0.38
